<a href="https://colab.research.google.com/github/Jaidon-Smith/AI-Karaoke/blob/main/AI_Forced_Alignment_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# intel-tensorflow is required to run on CPU
!pip install torch torchaudio soundfile omegaconf intel-tensorflow tensorflow_hub

     |████████████████████████████████| 7.6MB 11.4MB/s 
     |████████████████████████████████| 386.6MB 45kB/s 
     |████████████████████████████████| 645kB 42.1MB/s 
     |████████████████████████████████| 460kB 49.0MB/s 
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 2.3.0 which is incompatible.
ERROR: torchaudio 0.7.2 has requirement torch==1.7.1, but you'll have torch 1.7.0+cu101 which is incompatible.
ERROR: intel-tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.5 which is incompatible.
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0


In [2]:
#speech_file = 'speech_orig.wav'
speech_file = '/content/recorded.wav'


import os
import torch
import tensorflow as tf
import tensorflow_hub as tf_hub
from omegaconf import OmegaConf

language = 'en'

# load provided utils using torch.hub for brevity
_, decoder, utils = torch.hub.load('snakers4/silero-models', model='silero_stt', language=language)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils

# see available models
torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml', 'models.yml')
models = OmegaConf.load('models.yml')
available_languages = list(models.stt_models.keys())
assert language in available_languages

# load the actual tf model
tf_model = tf_hub.load(models.stt_models.get(language).latest.tf)

# download a single file, any format compatible with TorchAudio (soundfile backend)
torch.hub.download_url_to_file(models.stt_models.get(language).latest.meta.sample, dst = speech_file, progress=True)
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# tf inference
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(res.numpy())[0]))


Downloading: "https://github.com/snakers4/silero-models/archive/master.zip" to /root/.cache/torch/hub/master.zip
/usr/local/lib/python3.6/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
/usr/local/lib/python3.6/dist-packages/torchaudio/backend/utils.py:64: UserWarning: The interface of "soundfile" backend is planned to change in 0.8.0 to match that of "sox_io" backend and the current interface will be removed in 0.9.0. To use the new interface, do `torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False` before setting the backend to "soundfile". Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  'The interface of "soundfile" backend is planned to change in 0.8.0 


the birch canoe slit on the smooth planks blared the sheet to the dark blue background it's easy to tell a depth of a well four hours of steady work faced us


In [3]:
speech_file = '/content/recorded.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# tf inference
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(res.numpy())[0]))

the birch canoe slit on the smooth planks blared the sheet to the dark blue background it's easy to tell a depth of a well four hours of steady work faced us


In [4]:
batches

[['/content/recorded.wav']]

In [5]:
tf_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f412d864b00>

In [6]:
res

<tf.Tensor: shape=(1, 136, 999), dtype=float32, numpy=
array([[[9.9562091e-01, 2.1983167e-06, 4.2956420e-03, ...,
         7.4658174e-06, 6.9315388e-06, 1.2493891e-05],
        [2.3445152e-02, 1.3809526e-05, 9.7602135e-01, ...,
         3.1058439e-06, 1.8454997e-06, 3.6289552e-05],
        [2.6382593e-02, 6.0946192e-08, 8.4973122e-05, ...,
         1.2406194e-05, 1.3228365e-06, 9.7339523e-01],
        ...,
        [9.9961817e-01, 1.4285600e-06, 9.0514332e-06, ...,
         6.1206563e-05, 6.5136360e-06, 2.0248137e-05],
        [9.9961817e-01, 1.4284988e-06, 9.0513813e-06, ...,
         6.1204344e-05, 6.5132758e-06, 2.0246360e-05],
        [9.9961817e-01, 1.4285682e-06, 9.0528920e-06, ...,
         6.1196864e-05, 6.5128470e-06, 2.0243986e-05]]], dtype=float32)>

In [7]:
decoder

In [8]:
# construct a shape[1,136,999] tensor where the columns consist of a single 1 in one of the 136 and the others 0
# In this way I may find out what the 136 means

import numpy as np
value = 0
the_tensor = []
for i in range(136):
  if i == value:
    the_tensor.append(list(np.ones(999)))
  else:
    the_tensor.append(list(np.zeros(999)))

the_tensor = np.array(the_tensor)
the_tensor = the_tensor[np.newaxis, :]
#print(the_tensor.shape)
the_tensor = tf.convert_to_tensor(the_tensor)
print(the_tensor)


tf.Tensor(
[[[1. 1. 1. ... 1. 1. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(1, 136, 999), dtype=float64)


In [9]:
#res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(the_tensor.numpy())[0]))

# Test that the 999 does refer to time, and thus halving it will shorten the output
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(res.numpy())[0]))


the birch canoe slit on the smooth planks blared the sheet to the dark blue background it's easy to tell a depth of a well four hours of steady work faced us


In [10]:
res1, res2 = tf.split(res, [500,499],2)
print(res1.shape)
#print(decoder(torch.Tensor(res1.numpy())[0]))
'''---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
<ipython-input-38-617167afc803> in <module>()
      1 res1, res2 = tf.split(res, [500,499],2)
      2 print(res1.shape)
----> 3 print(decoder(torch.Tensor(res1.numpy())[0]))

1 frames
/root/.cache/torch/hub/snakers4_silero-models_master/utils.py in process(self, probs, wav_len, word_align)
     62     def process(self,
     63                 probs, wav_len, word_align):
---> 64         assert len(self.labels) == probs.shape[1]
     65         for_string = []
     66         argm = torch.argmax(probs, axis=1)

AssertionError: '''

(1, 136, 500)


'---------------------------------------------------------------------------\nAssertionError                            Traceback (most recent call last)\n<ipython-input-38-617167afc803> in <module>()\n      1 res1, res2 = tf.split(res, [500,499],2)\n      2 print(res1.shape)\n----> 3 print(decoder(torch.Tensor(res1.numpy())[0]))\n\n1 frames\n/root/.cache/torch/hub/snakers4_silero-models_master/utils.py in process(self, probs, wav_len, word_align)\n     62     def process(self,\n     63                 probs, wav_len, word_align):\n---> 64         assert len(self.labels) == probs.shape[1]\n     65         for_string = []\n     66         argm = torch.argmax(probs, axis=1)\n\nAssertionError: '

In [11]:
# construct a shape[1,136,999] tensore where the columns consist of a single 1 in one of the 136 and the others 0
# In this way I may find out what the 136 means

import numpy as np
value = 11
the_tensor = []
for i in range(999):
  if i == value:
    temp = list(np.ones(136))
    temp[10] = 0
    the_tensor.append(temp)
  else:
    temp = list(np.zeros(136))
    the_tensor.append(temp)
    if i == 12:
      temp[10] = 1


the_tensor = np.array(the_tensor)
the_tensor = np.swapaxes(the_tensor, 0, 1)
the_tensor = the_tensor[np.newaxis, :]
#print(the_tensor.shape)
the_tensor = tf.convert_to_tensor(the_tensor)
print(the_tensor)

print(decoder(torch.Tensor(the_tensor.numpy())[0]))

tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(1, 136, 999), dtype=float64)
entoen


In [12]:
#@title Install dependencies

!pip install -q omegaconf
!pip install -q torchaudio
!pip install -q soundfile
!pip install -q pydub

import os
from os.path import exists

if not exists('silero-models'):
  !git clone -q --depth 1 https://github.com/snakers4/silero-models

%cd silero-models

# silero imports
import torch
import random
from glob import glob
from omegaconf import OmegaConf
from utils import (init_jit_model, 
                   split_into_batches,
                   read_audio,
                   read_batch,
                   prepare_model_input)
from colab_utils import (record_audio,
                         audio_bytes_to_np,
                         upload_audio)

device = torch.device('cpu')   # you can use any pytorch device
models = OmegaConf.load('models.yml')

# imports for uploading/recording
import numpy as np
import ipywidgets as widgets
from scipy.io import wavfile
from IPython.display import Audio, display, clear_output
from torchaudio.functional import vad


     |████████████████████████████████| 776.8MB 25kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.7.1 which is incompatible.
/content/silero-models


In [13]:
sample_rate = 16000

clear_output()
audio = record_audio(10)
wavfile.write('/content/recorded.wav', sample_rate, (32767*audio).numpy().astype(np.int16))


#button = widgets.Button(description="Record Speech")
#button.on_click(_record_audio)
#display(button)

display(Audio(audio, rate=sample_rate, autoplay=True))
wavfile.write('test.wav', sample_rate, (32767*audio).numpy().astype(np.int16))
  

Starting recording for 10 seconds...


<IPython.core.display.Javascript object>

Finished recording!


In [14]:
len(decoder.labels)

999

In [15]:
decoder.blank_idx

0

In [16]:
speech_file = '/content/recorded.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)

'__________________           __   _______________________        _____               _change___          _s_________________'

In [17]:
result = np.argmax(res_array, axis=2)

#result = f(result)

In [19]:
result[0].tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 0,
 0,
 998,
 998,
 998,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 0,
 0,
 0,
 0,
 0,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 0,
 742,
 0,
 0,
 0,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 998,
 0,
 975,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [20]:
#Testing singing
speech_file = '/content/fireflies_voc_1min.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)

OSError: ignored

In [21]:
decoder.blank_idx

0

In [22]:
np.argsort(res_array, axis=2)[0][100]
#result = np.argmax(res_array, axis=2)
#result[0][100]

array([685, 822, 617, 653, 841, 821, 768, 933, 865, 833, 513, 940, 741,
       890, 905, 968, 777, 772, 456, 844, 306, 577, 720, 660, 705, 799,
       606, 882, 364, 761, 455, 541, 560, 950, 934, 717, 700, 919, 914,
       946, 901, 349, 351, 483, 942, 442, 673, 838, 793, 960, 495, 798,
       623, 837, 675, 764, 866, 936, 683, 562, 796, 362, 638, 828, 657,
       884, 938, 687, 342, 913, 707, 785, 883, 835, 780, 311, 855, 731,
       418, 808, 794, 912, 583, 877, 925, 691, 818, 923, 361, 920, 590,
       806, 394, 250, 561, 512, 792, 277, 847, 718, 504, 830, 471, 750,
       177, 614, 861, 664, 499, 400, 629, 557, 602, 903, 801, 628, 693,
       650, 384, 710, 681, 347, 420, 645, 856, 702, 642, 377, 379, 965,
       758, 586, 383, 454, 138, 737, 622, 514, 921, 726, 787, 609, 232,
       478, 162, 876, 763, 850, 459, 547, 797, 762, 676,  96, 745, 888,
       407, 966, 811, 635, 452, 630, 221, 714, 784, 131, 597, 875, 462,
       310, 527, 857, 749, 771, 337, 697, 217, 819, 449, 176, 61

In [25]:



#Testing singing with empty removed
speech_file = '/content/fireflies_voc_1min.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

original = result[0].copy()
for i,val in enumerate(original):
  #print(val)
  if val == 0:
    #print(np.argsort(res_array, axis=2)[0][i][-2])
    #result[0][i] = np.argsort(res_array, axis=2)[0][i][-2]
    result[0][i] = np.argpartition(res_array, -2, axis=2)[0][i][-2]

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)


"spspspspspspiispiiiiiiiii   ii  what   not     believeave  or      eyeseyes'eds    if  can   millillion   f ireire   ffliesesededss    my    up   the allall  as i   fell as  ly avevedveve    iiiiii22iiiiiiii222ii2222   becauseuse  they   foundll  they   open       airirirs     and leave    your drorops  everyevery   where    you   makeak  me   will   but    i  would  just   stand   and'nsss      starirss   sss iiiiiiiiiii22    i    like     make   mymymymy selfselfll   i inginginging i2222222222222    it   and   it a    i2222222ss  iiiii2sss2222    ssolowwve  ohohoh2 ohohoh22222i2222222   it   are   to say   that i   rathertherthersssss     why ingig      i m  less   lyeeingitiiii'2   becauseuse   everyevery  thth     is   nevernever  atactt   to   seeinginginginginging22222222222amanssssiiiiiiiiiii2    but   i  get   a  thousandthousands    hourouvesvesvesvessss   from   ten   thousandthousand    lightininenning    lovesvesves     is  they   tried  to teach   me    outout     d nsss 

In [94]:



#Testing singing with empty removed
speech_file = '/content/jepsen_voc.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

#original = result[0].copy()
#for i,val in enumerate(original):
  #print(val)
#  if val == 0:
    #print(np.argsort(res_array, axis=2)[0][i][-2])
    #result[0][i] = np.argsort(res_array, axis=2)[0][i][-2]
#    result[0][i] = np.argpartition(res_array, -2, axis=2)[0][i][-2]

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)


"____maybebe  's  right____________  maybebe there_  is  something  the  mat_ter with me_________________  i  _just   i don 't see how   world  that make   such  _wonderderful  things_____  could  be bad______  look  at  this  stuff_______  isn t  any  me_________  would     _think_    my_  _col_lect__ions_   comple__te_______  wouldn 't   _think_        girl________    _good_  who  _have____________   _every___thing________________  look_  at this_  _twelve____  tre_asas_ures_  and   all_______  _how   many  _won_dders_   can   want_  ca_verver__  who_____   look_      around_  you   you  think_________ _sure_________   she 's got_    _every___thingthing____________      get  getad  to  think  is    _   of plent_y_________ i 've got   who      from   what's it'  go _______________  you  want thing  over  us__________  i   got  _twent_y____________ _but_  who  _care__s____________  no_  big_  do_al_______________  i    want____  _________________________________________________________

In [24]:
decoder.labels

['_',
 'th',
 'the',
 'in',
 'an',
 're',
 'er',
 'on',
 'at',
 'ou',
 'is',
 'en',
 'to',
 'and',
 'ed',
 'al',
 'as',
 'it',
 'ing',
 'or',
 'of',
 'es',
 'ar',
 'he',
 'le',
 'st',
 'se',
 'om',
 'ic',
 'be',
 'we',
 'ly',
 'that',
 'no',
 'wh',
 've',
 'ha',
 'you',
 'ch',
 'ion',
 'il',
 'ent',
 'ro',
 'me',
 'id',
 'ac',
 'gh',
 'for',
 'was',
 'lo',
 'ver',
 'ut',
 'li',
 'ld',
 'ay',
 'ad',
 'so',
 'ir',
 'im',
 'un',
 'wi',
 'ter',
 'are',
 'with',
 'ke',
 'ge',
 'do',
 'ur',
 'all',
 'ce',
 'ab',
 'mo',
 'go',
 'pe',
 'ne',
 'this',
 'ri',
 'ght',
 'de',
 'one',
 'us',
 'am',
 'out',
 'fe',
 'but',
 'po',
 'his',
 'te',
 'ho',
 'ther',
 'not',
 'con',
 'com',
 'll',
 'they',
 'if',
 'ould',
 'su',
 'have',
 'ct',
 'ain',
 'our',
 'ation',
 'fr',
 'ill',
 'now',
 'sa',
 'had',
 'tr',
 'her',
 'per',
 'ant',
 'oun',
 'my',
 'ul',
 'ca',
 'by',
 'what',
 'red',
 'res',
 'od',
 'ome',
 'ess',
 'man',
 'ex',
 'she',
 'pl',
 'co',
 'wor',
 'pro',
 'up',
 'thing',
 'there',
 'ple',


In [88]:
import math

subproblems = []
def minimal_token(text, tokens):
  global subproblems
  subproblems = [None for i in range(len(text))]
  solution = minimal_token_recursive(text, 0, tokens)
  return solution

def minimal_token_recursive(text, index, tokens):
  global subproblems
  if index >= len(text):
    return (0,[])
  # If we haven't calculated the solution before, calculate it now
  if subproblems[index] is None:
    subproblems[index] = minimal_token_recursive_calculate(text, index, tokens)
    
  return subproblems[index]
  


def minimal_token_recursive_calculate(text, index, tokens):
  minimum = None
  minimum_token = None
  for i, token in enumerate(tokens):
    if str.find(text[index:], token) == 0:
      subproblem = minimal_token_recursive(text, index+len(token), tokens)
      if subproblem is None:
        raise Exception("Sub none")
      if minimum is None or subproblem[0] < minimum[0]:
        minimum = subproblem
        minimum_token = i
  if minimum == None:
    #raise Exception("This text cannot be represented")
    # or instead just return something with a large cost
    return (math.inf, [])
  
  new_list = minimum[1].copy()
  new_list.insert(0, minimum_token)
  return (1 + minimum[0], new_list)

  


In [90]:
#minimal_token("the inovation sensation", decoder.labels)
minimal_token("the", decoder.labels)

(1, [2])

In [33]:
decoder.

[None, None, None, None, None]

In [38]:
str.find("hello", "ll")

2

In [39]:
"hello"[0:]

'hello'

In [41]:
k = [1,2]
k.insert(2,0)
k

[1, 2, 0]

In [64]:
minimal_token("hello", ["he", "llo", "l", "lo", "o"])

he
he
llo
l
he
llo
l
he
llo
l
lo
o
1 ['o']
lo
o
1 ['lo']
lo
o
1 ['llo']
llo
l
lo
o
2 ['he', 'llo']


(2, ['he', 'llo'])

In [69]:
minimal_token("hello", ["he", "llo", "l", "lo"])

he
he
llo
l
he
llo
l
he
llo
l
lo
lo
1 ['lo']
lo
1 ['llo']
llo
l
lo
2 ['he', 'llo']


(2, ['he', 'llo'])

In [45]:
test = [1].append(k)
test

In [95]:
res.shape

TensorShape([1, 2453, 999])

In [93]:
the_string = '''look at this stuff isnt it neat wouldn't you think my collection's complete wouldn't you think i'm the girl girl who has everything'''

minimal_token(the_string, decoder.labels)

(64,
 [221,
  998,
  8,
  998,
  75,
  998,
  25,
  980,
  621,
  998,
  10,
  974,
  970,
  998,
  17,
  998,
  74,
  8,
  998,
  184,
  974,
  991,
  970,
  998,
  37,
  998,
  258,
  998,
  113,
  998,
  432,
  24,
  413,
  991,
  975,
  998,
  836,
  87,
  998,
  184,
  974,
  991,
  970,
  998,
  37,
  998,
  258,
  998,
  973,
  991,
  982,
  998,
  2,
  998,
  968,
  998,
  968,
  998,
  171,
  998,
  230,
  998,
  294,
  131])

In [157]:
import math


subproblems = []
def forced_alignment(correct_text, text_from_speech):
  global subproblems

  time_values = np.array(text_from_speech).shape[0]

  for i in range(len(correct_text)):
    subproblems.append([])
    for j in range(time_values):
      subproblems[-1].append(None)

  for i in range(len(correct_text)):
    for j in range(time_values):
      forced_alignment_recursive(correct_text, text_from_speech, i, j)

  
  solution = forced_alignment_recursive(correct_text, text_from_speech, len(correct_text) - 1, time_values - 1)
  return solution

def forced_alignment_recursive(correct_text, text_from_speech, text_index, time):
  global subproblems
  
  if time < text_index or text_index < 0:
    return (-math.inf, [])
  if time == 0:
    return (0, [0])
  

  # If we haven't calculated the solution before, calculate it now
  if subproblems[text_index][time] is None:
    subproblems[text_index][time] = forced_alignment_recursive_calculate(correct_text, text_from_speech, text_index, time)
    
  return subproblems[text_index][time]
  


def forced_alignment_recursive_calculate(correct_text, text_from_speech, text_index, time):
  text = correct_text[text_index]
  probability = math.log(text_from_speech[time][text])

  


  

  
  options = [forced_alignment_recursive(correct_text, text_from_speech, text_index, time - 1),
             forced_alignment_recursive(correct_text, text_from_speech, text_index - 1, time - 1),
  ]

  option_probabilities = [i[0] for i in options]
  options_index = np.argmax(option_probabilities)

  indexes = options[options_index][1].copy()
  indexes.append(text_index)
  return (options[options_index][0] + probability, indexes)







  '''minimum = None
  minimum_token = None
  for i, token in enumerate(tokens):
    if str.find(text[index:], token) == 0:
      subproblem = forced_alignment_recursive(text, index+len(token), tokens)
      if subproblem is None:
        raise Exception("Sub none")
      if minimum is None or subproblem[0] < minimum[0]:
        minimum = subproblem
        minimum_token = i
  if minimum == None:
    #raise Exception("This text cannot be represented")
    # or instead just return something with a large cost
    return (math.inf, [])
  
  new_list = minimum[1].copy()
  new_list.insert(0, minimum_token)
  return (1 + minimum[0], new_list)'''

  


In [96]:
test = [1,3]
np.argmax(test)

1

In [97]:
math.log(5)

1.6094379124341003

In [102]:
text = '''you would not believe your eyes if ten million fireflies lit up the world as i fell asleep cause they fill the open air and leave teardrops everywhere you'd think me rude but i would just stand and stare i'd like to make myself believe that planet earth turns slowly it's hard to say that i'd rather stay awake when i'm asleep cause everything is never as it seems cause i'd get a thousand hugs from ten thousand lightning bugs as they tried to teach me how to dance a foxtrot above my head'''

In [114]:
correct_text = minimal_token(text, decoder.labels)[1]

speech_file = '/content/fireflies_voc_1min.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
text_from_speech = np.array(res)[0]

In [115]:
text_from_speech.shape

(716, 999)

In [136]:
#forced_alignment_recursive(correct_text, text_from_speech, len(correct_text) - 1, time_values - 1)
#forced_alignment_recursive(correct_text, text_from_speech, 1, 1)
subproblems

[]

In [162]:
result = forced_alignment(correct_text, text_from_speech)
result = [decoder.labels[correct_text[i]] for i in result[1]]
print(result)

['you', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'would', ' ', ' ', ' ', 'not', ' ', ' ', ' ', ' ', ' ', 'believe', ' ', ' ', ' ', 'your', ' ', ' ', ' ', ' ', ' ', ' ', 'eyes', 'eyes', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'if', ' ', ' ', 'ten', ' ', ' ', ' ', 'mil', 'mil', 'li', 'on', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'fir', 'ef', 'li', 'es', 'es', 'es', ' ', 'li', 't', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'up', ' ', ' ', ' ', 'the', ' ', 'world', ' ', ' ', ' ', 'as', ' ', 'i', ' ', ' ', ' ', 'fe', 'll', ' ', 'as', 'as', 'le', 'le', 'ep', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'ca', 'use', ' ', ' ', 'they', ' ', ' ', ' ', 'fi', 'll', ' ', ' ', 'the', ' ', ' ', ' ', 'open', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'air', 'air', ' ', ' ', ' ', ' ', ' ', ' 

In [142]:
subproblems

KeyboardInterrupt: ignored

In [151]:
test = np.array(subproblems)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [145]:
test.shape

(247, 716)

In [152]:
test[10][10]

(-inf, [10])

In [148]:
text_from_speech

array([[9.9777907e-01, 1.0155298e-05, 3.0716743e-05, ..., 1.3978097e-05,
        7.5386620e-06, 1.6055184e-05],
       [9.9767643e-01, 1.0758812e-05, 1.6359705e-05, ..., 1.0408000e-05,
        5.4513293e-06, 3.1531268e-05],
       [9.9757701e-01, 1.0993117e-05, 2.0236279e-05, ..., 1.0957790e-05,
        5.8924015e-06, 2.8305192e-05],
       ...,
       [9.9007308e-01, 8.3693310e-05, 2.7314860e-05, ..., 7.7893361e-05,
        8.9203546e-05, 1.4436091e-04],
       [9.9237907e-01, 4.8827249e-05, 1.2536126e-04, ..., 1.0637233e-04,
        6.2168918e-05, 3.0697611e-05],
       [9.9616086e-01, 1.7994542e-05, 1.5976321e-04, ..., 1.7268624e-04,
        2.6818667e-05, 8.2173436e-05]], dtype=float32)